# Programación 201

# NBA player of the week

### Ejercicio 1

# Descripción y enunciado
El objetivo de esta actividad es la gestión y preparación de datos. La gestión de datos es una parte muy importante de cualquier proceso de minería de datos, ya que los datos se deben preparar y preprocesar antes de iniciar el proceso de extracción de conocimiento.
Esta tarea ilustra algunos de los procedimientos básicos con los que nos podemos encontrar para preparar los datos: analizar los tipos de datos, tranformar valores, detectar e eliminar outliers, hacer un muestreo, etcétera.

# Fichero de datos
El conjunto de datos con el que trabajaremos en esta actividad surge de la bases de datos disponibles en Kaggle . En concreto, los datos corresponden a los jugadores nombrados jugadores de la semana desde la temporada 1984-1985 hasta la temporada 2017-2018. Estos datos nos ofrecen múltiples posibilidades para consolidar los conocimientos y competencias de manipulación de datos, preprocesado y análisis descriptivo. El fichero `NBA_player_of_the_week.csv` se encuentra adjunto.


# Ingesta de datos
Actualmente el fichero se encuentra en la misma carpeta de datos RMD, está en adjunto. Se enviará nueva práctica similar con los datos de ingesta a través de scraping / API / base de datos para R y Python.


# Valoraciones
- se valorarán los trabajos por los detalles, no vale con el solo enviar un trozo de código, sino la explicación es fundamental
- explicar además los resumen de cada sección (carga, preparación, EDA, limpieza, nuevos atributos, discretización, split train-test)
- conclusiones y comparativas son los más importantes para comenzar con Data Mining.

### 1) Cargar un juego de datos 

`Librerías:`

In [1]:
import pandas as pd
import numpy as np

`Carga de datos:`

In [4]:
nba = pd.read_csv("NBA_player_of_the_week.csv")

### 2) Descripción de los datos

Describir brevemente la estructura de los datos: qué tipo de datos contiene, número de filas y columnas y nombre de las columnas. Realizar un resumen descriptivo de los datos.

In [5]:
# Tipo de datos que contiene
type(nba)

pandas.core.frame.DataFrame

In [18]:
# Observamos el tipo de dato por dimensión
nba.dtypes

Age                    int64
Conference            object
Date                  object
Draft Year             int64
Height                object
Player                object
Position              object
Season                object
Season short           int64
Seasons in league      int64
Team                  object
Weight                object
Real_value           float64
dtype: object

In [13]:
# Número de filas:
print("- El dataframe 'NBA_player_of_the_week' contiene: {} filas".format(len(nba)))
# Nombre de columnas:
print("- Nombres de las columnas : {}".format(nba.columns.values))

- El dataframe 'NBA_player_of_the_week' contiene: 1145 filas
- Nombres de las columnas : ['Age' 'Conference' 'Date' 'Draft Year' 'Height' 'Player' 'Position'
 'Season' 'Season short' 'Seasons in league' 'Team' 'Weight' 'Real_value']


In [24]:
# Estadísticas básicas: 
nba.describe()

,Conference,Date,Height,Player,Position,Season,Team,Weight
count,761,1145,1145,1145,1145,1145,1145,1145
unique,2,761,35,274,11,34,35,96
top,East,"Jan 12, 2003",6-9,LeBron James,G,2002-2003,Los Angeles Lakers,250
freq,381,3,129,61,178,47,71,106


### 3) Preparación de la base de datos
#### 3.1 Asignar NA a los valores ausentes en los datos de `Conference`.

In [26]:
# Reviso los valores ausentes por Variable
nba.isna().sum()

Age                    0
Conference           384
Date                   0
Draft Year             0
Height                 0
Player                 0
Position               0
Season                 0
Season short           0
Seasons in league      0
Team                   0
Weight                 0
Real_value             0
dtype: int64

In [40]:
# Reviso los valores que componen la variable Conference
nba.Conference.value_counts(dropna = False)

NaN     384
East    381
West    380
Name: Conference, dtype: int64

In [42]:
# Al no haber valores automáticamente se generean NaN , sin embargo voy a asignar el valor NA a cada uno de ellos:
nba.Conference.fillna("Na", inplace=True)

In [48]:
# Realizamos la comprobación de que la asignación ha sido correcta
print(nba.Conference.unique())

['Na' 'East' 'West']


#### 3.2 Transformar los datos de Conference en variable categórica y mostrar la tabla de frecuencia.

In [54]:
# Transformación de los datos de `Conference`
nba["Conference"] = nba.Conference.astype("category")
# Comprobamos la transformación
nba.Conference.dtypes

CategoricalDtype(categories=['East', 'Na', 'West'], ordered=False)

In [59]:
# Tabla de frecuencia
nba.Conference.value_counts()

Na      384
East    381
West    380
Name: Conference, dtype: int64

#### 3.3 Comprobar que la variable tiene xxx valores ausentes

In [64]:
# Recordemos que los valores ausentes fueron sustituidos por "Na"
nba.groupby(['Conference'])['Conference'].count()

Conference
East    381
Na      384
West    380
Name: Conference, dtype: int64

#### 3.4 Transformar las variables Height, Player, Position, Season, Team y Weight en variables categóricas.

In [65]:
nba["Height"] = nba.Height.astype("category")
nba["Player"] = nba.Player.astype("category")
nba["Position"] = nba.Position.astype("category")
nba["Season"] = nba.Season.astype("category")
nba["Team"] = nba.Team.astype("category")
nba["Weight"] = nba.Weight.astype("category")

#### 3.5 Comprobar que estas variables no tienen valores perdidos.

In [72]:
# Comprobamos que las variables transformadas no contengan valores perdidos tras realizar la suma de los elementos
nba[["Height", "Player", "Position", "Season", "Team", "Weight"]].isna().sum()

Height      0
Player      0
Position    0
Season      0
Team        0
Weight      0
dtype: int64

#### 3.6 Calcular el número de anyos que pasan entre que salio el jugador elegido en el draft y la temporada en la que fue jugador de la semana. Denomina la variable tiempo

In [75]:
nba["Tiempo"] = nba["Season short"] - nba["Draft Year"]

In [82]:
# Traemos las variables involucradas y comprobamos
nba[["Season short", "Draft Year", "Tiempo"]].head(10)

,Season short,Draft Year,Tiempo
0,1985,1978,7
1,1985,1982,3
2,1985,1979,6
3,1985,1969,16
4,1985,1978,7
5,1985,1980,5
6,1985,1982,3
7,1985,1981,4
8,1985,1979,6
9,1985,1982,3


####  3.7 ¿Cual es el número de años que más veces se repite entre que salen elegidos en el draft y se proclaman mejores jugadores? (moda de la variable tiempo)

In [94]:
# Podemos observar que el año que más se repite es:
nba.Tiempo.mode()

0    7
dtype: int64

#### 3.8 La altura de los jugadores está expresada en pies (aquellas con guión) y cm. Convertir la altura expresada en pies a centimetros (https://www.piesametros.info/](https://www.piesametros.info/). Posteriormente, eliminar el texto “cm” de los datos de tipo altura y convertir la variable en numérica. Calcular media, desviación típica y boxplot.

In [112]:
nba.Height.str.replace("-", ".").str.replace("cm", "")

0       6.5
1       6.6
2       6.6
3       7.2
4       6.9
       ... 
1140    196
1141    193
1142    211
1143    211
1144    196
Name: Height, Length: 1145, dtype: object

#### 3.9 El peso de los jugadores está expresada en libras y kg. Convertir el peso expresado en kgs en libras [https://www.metric-conversions.org/es/peso/kilogramos-a-libras.htm](https://www.metric-conversions.org/es/peso/kilogramos-a-libras.htm). Convertir la variable en numérica.Calcular media, desviación típica y boxplot.

In [110]:
nba_kg = nba.Weight.str.contains("kg")
# nba_kg["Weight"] = nba_kg.Weight.str.replace("kg", "")
type(nba_kg)

pandas.core.series.Series

In [104]:
nba["kg"] = nba.Weight.str.contains("kg")

In [111]:
nba["kg"]
type(nba["kg"])

pandas.core.series.Series

### 4 Información sobre posiciones en el campo y equipos.
#### 4.1 Fusionar las categorias de posición en el campo F,F-C y FC en la categoría genérica Forward. Fusionar las categorias G,G-F y GF en la categoria genérica Guard. Renombrar la categoría ‘PG’ como ‘Point Guard’, ‘SG’ como ‘Shooting Guard’, ‘SF’ como ‘Small Forward’, ‘PF’ como ‘Power Forward’ y ‘C’ como ‘Center’.

#### 4.2 ¿En que posición ha sido más veces los jugadores de la semana nombrados?

In [121]:
nba.Position.mode()

0    G
Name: Position, dtype: category
Categories (11, object): [C, F, F-C, FC, ..., PF, PG, SF, SG]

#### 4.3 ¿Cuántos equipos distintos hay?

In [116]:
nba.Team.describe()

count                   1145
unique                    35
top       Los Angeles Lakers
freq                      71
Name: Team, dtype: object

In [118]:
len(nba["Team"].unique())

35

#### 4.4 ¿Cuáles son los cinco equipos que más veces han sido nombrados sus jugadores como jugador de la semana?

In [120]:
(nba.Team.value_counts()).head(5)

Los Angeles Lakers     71
San Antonio Spurs      61
Cleveland Cavaliers    59
Miami Heat             57
Houston Rockets        56
Name: Team, dtype: int64

### 5 Información sobre jugadores. Estadísticos descriptivos.
#### 5.1 Crear una base datos denominada nba.unique en la que no se repita el nombre de ningún jugador.Trabajar a partir de ahora con esta base de datos.

In [127]:
nba_unique = nba.drop_duplicates("Player")

In [128]:
nba_unique.head()

,Age,Conference,Date,Draft Year,Height,Player,Position,Season,Season short,Seasons in league,Team,Weight,Real_value,Tiempo,kg
0,29,Na,"Apr 14, 1985",1978,6-5,Micheal Ray Richardson,PG,1984-1985,1985,6,New Jersey Nets,189,1.0,7,False
1,23,Na,"Apr 7, 1985",1982,6-6,Derek Smith,SG,1984-1985,1985,2,Los Angeles Clippers,205,1.0,3,False
2,28,Na,"Apr 1, 1985",1979,6-6,Calvin Natt,F,1984-1985,1985,5,Denver Nuggets,220,1.0,6,False
3,37,Na,"Mar 24, 1985",1969,7-2,Kareem Abdul-Jabbar,C,1984-1985,1985,15,Los Angeles Lakers,225,1.0,16,False
4,28,Na,"Mar 17, 1985",1978,6-9,Larry Bird,SF,1984-1985,1985,5,Boston Celtics,220,1.0,7,False


#### 5.2 Listar el nombre de los 10 jugadores más bajos que han sido nombrados alguna vez jugadores de la semana. No debe repetirse ningún nombre.

In [141]:
nba_unique.Height.sort_values().tail()

188    7-2
256    7-3
483    7-3
236    7-4
486    7-6
Name: Height, dtype: category
Categories (35, object): [175cm, 183cm, 185cm, 191cm, ..., 7-2, 7-3, 7-4, 7-6]

#### 5.3 Listar el nombre de los 10 jugadores más altos que han sido nombrados y mostrar su altura. No debe repetirse ningún nombre.

### 6 Eliminación de outliers.
#### 6.1 Eliminar los outliers de la variable peso (Weight) y guardar en un data frame nba.avg

#### 6.2 Comparar los boxplots de los datos originales en nba.unique y de los datos del peso de nba.avg

#### 6.3 Interpretar los resultados

### 7 Clasificación de jugadores
#### 7.1 Añadir una columna al conjunto de datos denominada “Altura” que contenga los valores: {“bajo”, “normal”, “alto”}. La asignación del valor depende de la altura de cada jugador. Se asigna:
- bajo: los 50 jugadores más bajos
- alto: los 50 jugadores más altos
- normal: el resto de jugadores

#### 7.2 Añadir una columna al conjunto de datos denominada “Peso” que contenga los valores: {“bajo”, “normal”, “alto”}. La asignación del valor depende del peso de cada jugador. Se asigna:
- bajo: los 50 jugadores con menor peso
- alto: los 50 jugadores con mayor peso
- normal: el resto de jugadores

#### 7.3 Calcular la media y desviación típica del peso (Weight) según la clasificación de altura de los jugadores.

#### 7.4 Realizar un boxplot del peso (Weight) según la clasificación de altura del jugador. Los boxplots deben estar ordenados de menos a mayor altura.

#### 7.5 Hacer un listado con el nombre de jugadores que tienen una altura ‘normal’ y un peso ‘alto’. Ordenar alfabéticamente por nombre.